In [1]:
import ROOT
import glob

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x65e4410


In [2]:
%jsroot on

In [3]:
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(1)
ROOT.ildStyle.SetPalette(ROOT.kBird)

In [5]:
%%cpp
using namespace ROOT::VecOps;

In [6]:
ROOT.gInterpreter.Declare("#include <podio/DataSource.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/dataframe.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/kinematics.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/vector_utils.h>")
ROOT.gSystem.Load("libedm4hep")
_edm  = ROOT.edm4hep.ReconstructedParticleData()

In [7]:
# plots = False
plots = True
all = False
# all = True
df = None
# df = ROOT.RDataFrame("events", "data/truejet/test_dst/eLpR_15176_0.truejet.edm4hep.root")
# df = ROOT.podio.CreateDataFrame("data/truejet/test_dst/eLpR_15176_0.truejet.edm4hep.root")
if all:
    paths = glob.glob("/eos/user/l/lreichen/paris/*.edm4hep.root")
    files = [f"root://eosuser.cern.ch/{path}" for path in paths]
    df = ROOT.podio.CreateDataFrame(files)
else:
    df = ROOT.podio.CreateDataFrame("root://eosuser.cern.ch//eos/user/l/lreichen/paris/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500105.P4f_sw_sl.eL.pL.n000.d_dstm_15064_0.miniDST.edm4hep.root")

In [8]:
df.Describe()

Dataframe from datasource PODIO Datasource

Property                Value
--------                -----
Columns in total           99
Columns from defines        0
Event loops run             0
Processing slots            1

Column                                          Type                                            Origin
------                                          ----                                            ------
BCalRecoParticle                                edm4hep::ReconstructedParticleCollection        Dataset
BCalRecoParticle_startVertices                  edm4hep::VertexRecoParticleLinkCollection       Dataset
EventHeader                                     edm4hep::EventHeaderCollection                  Dataset
FinalColourNeutrals                             edm4hep::ReconstructedParticleCollection        Dataset
FinalColourNeutrals_PID_TrueJet_fafpf           edm4hep::ParticleIDCollection                   Dataset
FinalColourNeutrals_PID_TrueJet_fafpf_jet_00    e

In [9]:
%%cpp
ROOT::Math::PxPyPzEVector make_PxPyPzEVec(edm4hep::Vector3f P, float E)
{
    return ROOT::Math::PxPyPzEVector(P.x, P.y, P.z, E);
}

In [10]:
from math import sin
# stuff that could be arguments or read from file parameters
sqrt_s = 250 # GeV
x_angle = 0.014 # rad

p_x = 250 * sin(x_angle/2)

# TODO rethink if this is needed or to be done in this file/stage

In [11]:
%%cpp
// returns the first particle whose absolute type and pdg values are fulfilling type and pdg
edm4hep::ReconstructedParticle getParticleWithPID(const edm4hep::ParticleIDCollection& pid_col, std::int32_t type, std::int32_t pdg)
{
    for (const edm4hep::ParticleID& pid : pid_col) {
        if (abs(pid.getType()) == type && abs(pid.getPDG()) == pdg) {
            return pid.getParticle();
        }
    }
    return edm4hep::ReconstructedParticle();
}


In [ ]:
%%cpp
std::array<edm4hep::ReconstructedParticle, 5> getTrueJets(const edm4hep::ParticleIDCollection& tj_pid_col)
{
    edm4hep::ReconstructedParticle e = getParticleWithPID(tj_pid_col, 2, 11);
    edm4hep::ReconstructedParticle nu = getParticleWithPID(tj_pid_col, 2, 12);

    std::vector<edm4hep::ReconstructedParticle> isr;
    for (const auto& pid : tj_pid_col) {
        if (abs(pid.getType()) == 4) {
            isr.push_back(pid.getParticle());
        }
    }

    edm4hep::ReconstructedParticle isr1 = isr[0];
    edm4hep::ReconstructedParticle isr2 = isr[1];

    edm4hep::ReconstructedParticle overlay = getParticleWithPID(tj_pid_col, 5, 0);

    return std::array<edm4hep::ReconstructedParticle, 5>{e, nu, isr1, isr2, overlay};
}


In [13]:
%%cpp
ROOT::VecOps::RVec<edm4hep::ReconstructedParticle> getQuarkTrueJets(const edm4hep::ParticleIDCollection& icn_pid_col)
{
    RVec<edm4hep::ReconstructedParticle> res;
    for (const auto& pid : icn_pid_col) {
        if (pid.getType() == 1 || pid.getType() == 3) {
            // should only happen once anyway
            // fingers crossed
            auto particles = pid.getParticle().getParticles();
            return RVec<edm4hep::ReconstructedParticle>(particles.begin(), particles.end());
        }
    }
    return res;
}

In [14]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticle> getEnergySafeMCParticles(const edm4hep::RecoMCParticleLinkCollection& links,const edm4hep::ReconstructedParticle& reco)
{
    RVec<edm4hep::MCParticle> res;
    for (const auto& link : links) {
        if (link.getFrom() == reco && link.getWeight() == 1) {
            res.push_back(link.getTo());
        }
    }
    return res;
}

In [15]:
# get truejets for: e, nu?, q, qbar, isr1, isr2, overlay
# get their corresponding MC particles (straightforward?)
# or do I use ICN/FCN?
# TrueJetMCParticleLink with weight == 1 should be the correct thing according to documentation

# ugh cannot cast collections to RVec
# df = df.Define("e", "RVec(TrueJets_PID_TrueJetPID)[abs(RVec(TrueJets_PID_TrueJetPID.PDG())) == 11]")
# df = df.Define("e", "getParticleWithPID(TrueJets_PID_TrueJetPID, 2, 11)")
# df = df.Define("tj_pfos", "std::cout << rdfentry_ << std::endl; return getTrueJets(TrueJets_PID_TrueJetPID);")
df = df.Define("tj_pfos", "getTrueJets(TrueJets_PID_TrueJetPID)")
df = df.Define("tj_quarks", "getQuarkTrueJets(InitialColourNeutrals_PID_TrueJet_fafpi)")

simple_tjs = ["e", "nu", "isr1", "isr2", "ovl"]

for i, tj in enumerate(simple_tjs):
    df = df.Define(tj, f"tj_pfos[{i}]")
    df = df.Define(f"{tj}_lvec", f"edm4hep::utils::p4({tj}, edm4hep::utils::UseEnergy)")
    df = df.Define(f"{tj}_mc", f"getEnergySafeMCParticles(TrueJetMCParticleLink, {tj})")
    df = df.Define(f"{tj}_mc_lvecs", f"return Map({tj}_mc, [] (const auto& el) {{return edm4hep::utils::p4(el, edm4hep::utils::UseEnergy);}});")
    df = df.Define(f"{tj}_mc_sum_lvec", f"Sum({tj}_mc_lvecs, ROOT::Math::PxPyPzEVector())")

# for the quark truejets everything needs one more loop because their number can vary
df = df.Define("had_lvecs", "return Map(tj_quarks, [] (const auto& el) {return edm4hep::utils::p4(el, edm4hep::utils::UseEnergy);});")
df = df.Define("had_sum_lvec", "Sum(had_lvecs, ROOT::Math::PxPyPzEVector())")
df = df.Define("had_mc", "return Map(tj_quarks, [&] (const auto& el) {return getEnergySafeMCParticles(TrueJetMCParticleLink, el);})")
# df = df.Define("had_mc_acc", "std::accumulate(had_mc.begin(), had_mc.end(), RVec<edm4hep::MCParticle>(), Concatenate)")
# df = df.Define("had_mc_acc", "RVec<edm4hep::MCParticle> res; for (const auto& vec : had_mc) {res.insert(res.end(), vec.begin(), vec.end());} return res;")
df = df.Define("had_mc_acc_lvecs", """
                RVec<ROOT::Math::PxPyPzEVector> res;
                for (const auto& vec : had_mc) {
                    ROOT::Math::PxPyPzEVector lvec;
                    // sum up all mcps that make up one reco
                    for (const auto& mcp : vec) {
                        auto lv = edm4hep::utils::p4(mcp, edm4hep::utils::UseEnergy);
                        lvec += lv;
                    }
                    res.push_back(lvec);
                }
                return res;
                """)
# df = df.Define("had_mc_acc_lvecs", "return Map(had_mc_acc, [] (const auto& el) {return edm4hep::utils::p4(el, edm4hep::utils::UseEnergy);})")
df = df.Define(f"had_mc_acc_sum_lvec", f"Sum(had_mc_acc_lvecs, ROOT::Math::PxPyPzEVector())")

df = df.Define("had_lvecs_px", "return Map(had_lvecs, [] (const auto& el) {return el.Px();})")
df = df.Define("had_lvecs_py", "return Map(had_lvecs, [] (const auto& el) {return el.Py();})")
df = df.Define("had_lvecs_pz", "return Map(had_lvecs, [] (const auto& el) {return el.Pz();})")
df = df.Define("had_lvecs_e",  "return Map(had_lvecs, [] (const auto& el) {return el.E();})")

df = df.Define("had_mc_acc_lvecs_px", "return Map(had_mc_acc_lvecs, [] (const auto& el) {return el.Px();})")
df = df.Define("had_mc_acc_lvecs_py", "return Map(had_mc_acc_lvecs, [] (const auto& el) {return el.Py();})")
df = df.Define("had_mc_acc_lvecs_pz", "return Map(had_mc_acc_lvecs, [] (const auto& el) {return el.Pz();})")
df = df.Define("had_mc_acc_lvecs_e",  "return Map(had_mc_acc_lvecs, [] (const auto& el) {return el.E();})")

# df = df.Define("e_lvec", "ROOT::Math::PxPyPzEVector(e.getMomentum().x, e.getMomentum().y, e.getMomentum().z, e.getEnergy())")
# df = df.Define("e_lvec", "edm4hep::utils::p4(e, edm4hep::utils::UseEnergy)")
# df = df.Define("e_mc", "getEnergySafeMCParticles(TrueJetMCParticleLink, e)")
# df = df.Define("e_mc_lvecs", "return Map(e_mc, [] (const auto& el) {auto energy = el.getEnergy(); auto p = el.getMomentum(); return ROOT::Math::PxPyPzEVector(p.x, p.y, p.z, energy);};)")
# df = df.Define("e_mc_lvecs", "Map(e_mc, edm4hep::utils::p4)")
# df = df.Define("e_mc_lvecs", "return Map(e_mc, [] (const auto& el) {return edm4hep::utils::p4(el, edm4hep::utils::UseEnergy);});")
# df = df.Define("e_mc_sum_lvec", "Sum(e_mc_lvecs, ROOT::Math::PxPyPzEVector())")

df = df.Define("e_mc_charge", "return Sum(Map(e_mc, [] (const auto& el) {return el.getCharge();}))")
df = df.Define("e_charge", "e.getCharge()")

In [16]:
df = df.Define("e_energy", "e_lvec.energy()")
df = df.Define("nu_energy", "nu_lvec.energy()")
df = df.Define("isr1_energy", "isr1_lvec.energy()")
df = df.Define("isr2_energy", "isr2_lvec.energy()")
df = df.Define("ovl_energy", "ovl_lvec.energy()")
df = df.Define("had_energy", "had_sum_lvec.energy()")

df = df.Define("e_mc_sum_energy", "e_mc_sum_lvec.energy()")
df = df.Define("nu_mc_sum_energy", "nu_mc_sum_lvec.energy()")
df = df.Define("isr1_mc_sum_energy", "isr1_mc_sum_lvec.energy()")
df = df.Define("isr2_mc_sum_energy", "isr2_mc_sum_lvec.energy()")
df = df.Define("ovl_mc_sum_energy", "ovl_mc_sum_lvec.energy()")
df = df.Define("had_mc_sum_energy", "had_mc_acc_sum_lvec.energy()")

df = df.Define("e_energy_diff", "e_mc_sum_energy - e_energy")

In [17]:
h_e = df.Histo1D(("", "; reco energy", 150, 0., 150.), "e_energy")
h_e_mc = df.Histo1D(("", "; true energy", 150, 0., 150.), "e_mc_sum_energy")
h_e_diff = df.Histo1D(("", "; true-reco energy", 300, -40., 40.), "e_energy_diff")

h_nu = df.Histo1D(("", "; reco energy", 150, 0., 150.), "nu_energy")
h_nu_mc = df.Histo1D(("", "; true energy", 150, 0., 150.), "nu_mc_sum_energy")
h_isr1 = df.Histo1D(("", "; reco energy", 150, 0., 150.), "isr1_energy")
h_isr1_mc = df.Histo1D(("", "; true energy", 150, 0., 150.), "isr1_mc_sum_energy")
h_isr2 = df.Histo1D(("", "; reco energy", 150, 0., 150.), "isr2_energy")
h_isr2_mc = df.Histo1D(("", "; true energy", 150, 0., 150.), "isr2_mc_sum_energy")
h_ovl = df.Histo1D(("", "; reco energy", 150, 0., 150.), "ovl_energy")
h_ovl_mc = df.Histo1D(("", "; true energy", 150, 0., 150.), "ovl_mc_sum_energy")
h_had = df.Histo1D(("", "; reco energy", 150, 0., 150.), "had_energy")
h_had_mc = df.Histo1D(("", "; true energy", 150, 0., 150.), "had_mc_sum_energy")

In [18]:
if plots:
    c_e = ROOT.TCanvas()
    h_e.Draw()
    c_e.Draw()

    c_e_mc = ROOT.TCanvas()
    h_e_mc.Draw()
    c_e_mc.Draw()

    # c_e_diff = ROOT.TCanvas()
    # f = ROOT.TF1(f"f", "gaus", -2., 1.5)
    # # h_e_diff.Fit(f, "R")
    # h_e_diff.Draw()
    # c_e_diff.Draw()

    c_nu = ROOT.TCanvas()
    h_nu.Draw()
    c_nu.Draw()

    c_nu_mc = ROOT.TCanvas()
    h_nu_mc.Draw()
    c_nu_mc.Draw()

    c_isr1 = ROOT.TCanvas()
    h_isr1.Draw()
    c_isr1.Draw()

    c_isr1_mc = ROOT.TCanvas()
    h_isr1_mc.Draw()
    c_isr1_mc.Draw()

    c_isr2 = ROOT.TCanvas()
    h_isr2.Draw()
    c_isr2.Draw()

    c_isr2_mc = ROOT.TCanvas()
    h_isr2_mc.Draw()
    c_isr2_mc.Draw()

    c_ovl = ROOT.TCanvas()
    h_ovl.Draw()
    c_ovl.Draw()

    c_ovl_mc = ROOT.TCanvas()
    h_ovl_mc.Draw()
    c_ovl_mc.Draw()

    c_had = ROOT.TCanvas()
    h_had.Draw()
    c_had.Draw()

    c_had_mc = ROOT.TCanvas()
    h_had_mc.Draw()
    c_had_mc.Draw()


In [19]:
comps = ["px", "py", "pz", "e"]
branches = [f"{tj}_lvec" for tj in simple_tjs] + [f"{tj}_mc_sum_lvec" for tj in simple_tjs] + [f"had_lvecs_{c}" for c in comps] + [f"had_mc_acc_lvecs_{c}" for c in comps] + ["e_charge", "e_mc_charge"] #+ ["had_lvecs"]
if all:
    df.Snapshot("events", "data/post_paris/nano/all.root", branches)
else:
    df.Snapshot("events", "data/post_paris/nano/test.root", branches)